In [1]:
import sys
sys.path.insert(0, r"K:/Thesis/codes/crypto_project")
sys.path.insert(0, r"E:/Thesis/crypto_project")
import os
import sqlite3
import pandas as pd
import numpy as np
from tqdm import tqdm

DATABASE_LOCATION = r"E:/Thesis/database"
from factor_model.model_update.database_generators import (
    RAW_DATA_DB,
    RETURN_DB,
    FACTOR_MODEL_ESTIMATES
)
from typing import Dict


In [14]:
"".rjust(max(3 - str(val)[::-1].find("."), 0), '0')

'00'

In [2]:
from factor_model.risk_calculations.factor_covariance import (
    get_factor_return_correlation,
    get_factor_return_standard_deviation,
    assemble_factor_covariance_matrix
)
from factor_model.risk_calculations.specific_risk import (
    generate_raw_specific_risk,
    generate_raw_portfolio_specific_risk
)
from factor_model.risk_calculations.core_universe_portfolio import generate_market_portfolio

#### 0. Parameters

In [3]:
# A sample portfolio to test calculations and restrict query...
portfolio_details = {
    "BCUBE-USD": 100,
    "LNC-USD": 20,
    "PMG24050-USD": 30,
    "ZNN-USD": 50,
    "EFI-USD": 50,
    "BTC-USD": 150
}

# risk calculation
risk_calculation_parameters = {
    "correlation_half_life": 730,  # days
    "variance_half_life": 365,  # days
    "specific_risk_half_life": 365,
    "date": "2023-03-04",
    "minimum_history_spec_ret": 730
}

#### 1. Load the factor returns

In [4]:
with sqlite3.connect(os.path.join(DATABASE_LOCATION, FACTOR_MODEL_ESTIMATES)) as conn:
    factor_returns = pd.read_sql_query("SELECT * FROM factor_returns", conn)

#### 2. factor covariance related calculations

In [5]:
correlation = get_factor_return_correlation(factor_returns, risk_calculation_parameters)
display(correlation)
std = get_factor_return_standard_deviation(factor_returns, risk_calculation_parameters)
display(std)
covariance = assemble_factor_covariance_matrix(std, correlation)
display(covariance)


market  momentum  new_coin  reversal      size    volume
1765 market    1.000000  0.020237 -0.227545 -0.048430  0.303360 -0.730431
     momentum  0.020237  1.000000  0.082769 -0.265184 -0.097476  0.053568
     new_coin -0.227545  0.082769  1.000000  0.104668  0.037091  0.406697
     reversal -0.048430 -0.265184  0.104668  1.000000  0.027737  0.041279
     size      0.303360 -0.097476  0.037091  0.027737  1.000000 -0.592027
     volume   -0.730431  0.053568  0.406697  0.041279 -0.592027  1.000000

market      0.038177
momentum    0.006918
new_coin    0.006413
reversal    0.011822
size        0.008365
volume      0.009609
dtype: float64

,market,momentum,new_coin,reversal,size,volume
market,0.001458,0.000005,-0.000056,-0.000022,0.000097,-0.000268
momentum,0.000005,0.000048,0.000004,-0.000022,-0.000006,0.000004
new_coin,-0.000056,0.000004,0.000041,0.000008,0.000002,0.000025
reversal,-0.000022,-0.000022,0.000008,0.000140,0.000003,0.000005
size,0.000097,-0.000006,0.000002,0.000003,0.000070,-0.000048
volume,-0.000268,0.000004,0.000025,0.000005,-0.000048,0.000092


#### 3. specific risk modeling

In [6]:
with sqlite3.connect(os.path.join(DATABASE_LOCATION, FACTOR_MODEL_ESTIMATES)) as conn:
    specific_returns = pd.read_sql_query(f"SELECT * FROM specific_returns where ticker in {tuple(portfolio_details.keys())}", conn)

In [7]:
# raw specific risk calculation, based on a naive approach
spec_std, hist_len = generate_raw_specific_risk(specific_returns, risk_calculation_parameters, portfolio_details)

In [8]:
spec_std

{'BCUBE-USD': 0.10966703295406163,
 'LNC-USD': 0.14962437646714322,
 'PMG24050-USD': None,
 'ZNN-USD': 0.11430259704561414,
 'EFI-USD': 0.049052398563265764,
 'BTC-USD': 0.008626331234568763}

In [9]:
hist_len

{'BCUBE-USD': 612,
 'LNC-USD': 989,
 'PMG24050-USD': 0,
 'ZNN-USD': 1360,
 'EFI-USD': 576,
 'BTC-USD': 1766}

In [10]:
generate_raw_portfolio_specific_risk(spec_std, portfolio_details)

0.03519495140174019